In [1]:
!pip install pandas
!pip install optuna
!pip install optuna-dashboard
!pip install scikit
!pip install imbalanced-learn
from sklearn.preprocessing import (
    MaxAbsScaler,
    MinMaxScaler,
    Normalizer,
    PowerTransformer,
    QuantileTransformer,
    RobustScaler,
    StandardScaler,
    minmax_scale,
)
from sklearn.metrics import recall_score, accuracy_score,f1_score, precision_score, roc_auc_score
from sklearn.model_selection import train_test_split, StratifiedKFold
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.ensemble import IsolationForest
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
import warnings
import optuna
import pandas as pd
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, roc_auc_score

ERROR: Could not find a version that satisfies the requirement scikit (from versions: none)
ERROR: No matching distribution found for scikit


c:\Users\TAN LE ZHAN\Documents\GitHub\ADL\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using", device)
def init_weights(model):
    if isinstance(model, nn.Linear):  # Apply only to linear layers
        # He initialization (recommended for ReLU activations)
        # print("Initializing weights using kaiming")
        nn.init.kaiming_normal_(model.weight, mode='fan_in', nonlinearity='relu')
        
        # Bias initialization (zero initialization is fine)
        if model.bias is not None:
            nn.init.zeros_(model.bias)
def fold_to_dataloader_tensor(train_x, test_x, train_y, test_y, batch_size=64, device=device):
    train_dataset = TensorDataset(
        torch.tensor(train_x.values,dtype=torch.float32).to(device), 
        torch.tensor(train_y.values,dtype=torch.float32).to(device))
    val_dataset = TensorDataset(
        torch.tensor(test_x.values,dtype=torch.float32).to(device), 
        torch.tensor(test_y.values,dtype=torch.float32).to(device))

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    val_loader = DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=True, drop_last=True)
    return train_loader, val_loader 

def get_feature_count(loader):
    """returns the number of features in the dataset"""
    return next(iter(loader))[0].shape[1]

from Criterion_Models import *
def criterion_mapping(criterion_choice:str, pos_weight:float=None, alpha:float=None, gamma:float=None):
    """
    Feel free to add any custom loss functions here.
    returns function for criterion
    """
    if criterion_choice == "FocalLoss":
        return FocalLoss(alpha =alpha, gamma=gamma)
    elif criterion_choice == "DiceLoss":
        return DiceLoss()
    elif criterion_choice == "BCEWithLogitsLoss":
        return nn.BCEWithLogitsLoss(pos_weight=torch.tensor([pos_weight])) if pos_weight else nn.BCEWithLogitsLoss()
    return nn.BCEWithLogitsLoss() 

def augment_data_in_place(X, X_test, Y=None, normalisation_method=MinMaxScaler(), noise=None):
    all_numerical_columns = ['Age', 'UAlb', 'Ucr', 'UACR', 'TC', 'TG', 'TCTG', 'LDLC', 'HDLC', 'Scr', 'BUN', 'FPG', 'HbA1c', 'Height', 'Weight', 'BMI', 'Duration']
    binary_columns = ['Gender', 'DR', 'Community_baihe', 'Community_chonggu', 'Community_huaxin', 'Community_jinze', 'Community_liantang', 'Community_xianghuaqiao', 'Community_xujin', 'Community_yingpu', 'Community_zhaoxian', 'Community_zhujiajiao']
    
    existing_columns = [col for col in all_numerical_columns if col in X.columns and col in X_test.columns]

    if not existing_columns:
        print("No matching columns found for augmentation. Normalised data only.")
        X = normalisation_method.fit_transform(X)
        X_test = normalisation_method.transform(X_test)
        return X, X_test

    X_copy = X.copy()
    X_test_copy = X_test.copy()
    
    # Log-transform
    X_copy.loc[:, existing_columns] = X_copy.loc[:, existing_columns].apply(np.log1p)
    X_test_copy.loc[:, existing_columns] = X_test_copy.loc[:, existing_columns].apply(np.log1p)

    # Add noise ONLY to negatives (class 0) if Y is provided and noise is set
    if noise and noise > 0:
        if Y is None:
            raise ValueError("Y must be provided if noise is being added selectively.")
        # Identify negative class indices (class 0)
        negative_indices = Y[Y.iloc[:, 0] == 0].index
        noise_matrix = np.random.normal(0, noise, X_copy.loc[negative_indices, existing_columns].shape)
        X_copy.loc[negative_indices, existing_columns] += noise_matrix

    # Scale
    scaler = normalisation_method
    X_copy.loc[:, existing_columns] = scaler.fit_transform(X_copy.loc[:, existing_columns])
    X_test_copy.loc[:, existing_columns] = scaler.transform(X_test_copy.loc[:, existing_columns])

    return X_copy, X_test_copy


def iso_forest(X_train, Y_train, contamination=None, random_state=42):
    # print("Original\n", X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)
    X_train_cleaned, Y_train_cleaned = X_train.copy(), Y_train.copy()
    
    X_train_zeros = X_train[Y_train.iloc[:, 0] == 0]
    X_train_ones = X_train[Y_train.iloc[:, 0] == 1]
    Y_train_zeros = Y_train[Y_train.iloc[:, 0] == 0]
    Y_train_ones = Y_train[Y_train.iloc[:, 0] == 1] 
    # print("Ones and zeros\n", X_train_zeros.shape, Y_train_zeros.shape, X_train_ones.shape, Y_train_ones.shape)
    #only class 0s
    if X_train_zeros.isna().any().any():
        print("got NaN values in the training set")
    
    # Apply Isolation Forest to majority class only
    iso_forest = IsolationForest(contamination=contamination, random_state=random_state)
    try:
        outliers = iso_forest.fit_predict(X_train_zeros)
    except UserWarning as e:
        print("Caught warning during IsolationForest fitting:", e)
        outliers = np.ones(len(X_train_zeros))  # If warning occurs, keep all data
    # Keep only non-outlier majority samples
    X_train_zeros = X_train_zeros[outliers == 1]
    Y_train_zeros = Y_train_zeros[outliers == 1]
    # print("After iso:\n", X_train_zeros.shape, Y_train_zeros.shape, X_train_ones.shape, Y_train_ones.shape)
    
    # Combine the cleaned majority class with the untouched minority class
    X_train_cleaned = pd.concat([X_train_zeros, X_train_ones])
    Y_train_cleaned = pd.concat([Y_train_zeros, Y_train_ones])
    return X_train_cleaned, Y_train_cleaned


Using cuda


In [3]:
random_state = 42
raw_dataset = pd.read_csv("./data/processed_data.csv") #data has X and Y
X = raw_dataset.drop(columns=["DR"])
Y = pd.DataFrame(raw_dataset["DR"])
# Slice your data


X_FOR_FOLDS, X_FINAL_TEST, Y_FOR_FOLDS, Y_FINAL_TEST = train_test_split(X, Y, test_size=0.1, random_state=random_state, stratify=Y)

In [4]:
def FOLDS_GENERATOR(X, Y, normalisation_method=MinMaxScaler(), n_splits=5, random_state=None, oversampler=None, contamination=0.05, noise = None):
    """
    Generates stratified folds with specified normalization.
    normalisation_method should be an instance of a scaler, e.g.,
    - MinMaxScaler()
    Returns a list of tuples, each containing:
    (X_train_scaled, X_test_scaled, Y_train, Y_test), representing data for each fold
    """
    kF = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    kFolds_list = []

    for fold, (train_idx, test_idx) in enumerate(kF.split(X, Y)):
        # Split the data into training and testing sets for this fold
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        Y_train, Y_test = Y.iloc[train_idx], Y.iloc[test_idx]

        # print("Original\n", X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)
        X_train_cleaned, Y_train_cleaned = X_train.copy(), Y_train.copy()
        if contamination is not None and contamination > 0: #? using contamination = 0.0 works
            X_train_cleaned, Y_train_cleaned = iso_forest(X_train, Y_train, contamination=contamination, random_state=random_state)
        
        #? data augmentation on leftover data
        X_train_scaled, X_test_scaled = augment_data_in_place(X_train_cleaned, X_test, Y_train_cleaned,normalisation_method=normalisation_method, noise = noise)
        
        # Handle oversampling if needed
        #! use X_train_scaled and Y_train_cleaned for oversampling becasue y_train_cleaned no changes after augmentation
        # print("Before oversampling class distribution:")
        # print(Y_train_cleaned.value_counts())
        if oversampler:
            X_train_scaled, Y_train_cleaned = oversampler.fit_resample(X_train_scaled, Y_train_cleaned)
        # print("\nAfter oversampling class distribution:")
        # print(Y_train_cleaned.value_counts())
        # Convert scaled data back to DataFrame with the correct column names
        X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train_cleaned.columns)
        X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

        # Handle community columns
        community_cols = [col for col in X_train_scaled.columns if col.startswith('Community')]
        if community_cols:
            X_train_scaled[community_cols] = X_train_scaled[community_cols].apply(
                lambda row: pd.Series(np.eye(len(row))[row.argmax()]), axis=1
            ).set_axis(community_cols, axis=1)
        # print(X_train_scaled[community_cols].describe())

        # Ensure 'Gender' is still binary (0 or 1)
        if 'Gender' in X_train_scaled.columns:
            X_train_scaled['Gender'] = (X_train_scaled['Gender'] > 0.5).astype(int)
            X_test_scaled['Gender'] = (X_test_scaled['Gender'] > 0.5).astype(int)

        # Append the processed fold to the list
        kFolds_list.append((X_train_scaled, X_test_scaled, Y_train_cleaned, Y_test))

        print(f"Fold: {fold+1}, Train: {X_train_scaled.shape}, Test: {X_test_scaled.shape}")

    return kFolds_list

In [15]:
def train_and_evaluate(model, criterion, optimiser, scheduler, train_loader, val_loader, epochs=20, patience=5, device=device, threshold = 0.5):
    # if isinstance(model.last_layer(), nn.Sigmoid) and isinstance(criterion, nn.BCEWithLogitsLoss):
    #     raise ValueError("Model output is Sigmoid but criterion is BCEWithLogitsLoss. Please check your model and criterion compatibility.")
    best_val_loss = float('inf')
    best_model_state = None
    wait = 0
    criterion.to(device) #? Move criterion to device
    #* Epoch Training loop for this fold
    for epoch in range(1,epochs+1):
        #* Set model to training mode: essential for dropout and batch norm layers
        model.train()
        running_loss = 0.0 #? loss for this epoch
        #* Mini-batch training loop
        for batch, (inputs, labels) in enumerate(train_loader,start=1):
            optimiser.zero_grad() #? Zero the gradients
            assert not torch.isnan(inputs).any(), "Input has NaNs"
            assert not torch.isinf(inputs).any(), "Input has Infs"
            outputs = model(inputs) #? Forward pass through the model
            assert not torch.isnan(outputs).any(), "Model output has NaNs"
            assert not torch.isinf(outputs).any(), "Model output has Infs"
            loss = criterion(outputs, labels) #? Calculate loss
            assert not torch.isnan(loss).any(), "Model loss has NaNs"
            loss.backward() #? Backpropagation
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            running_loss += loss.item()
            optimiser.step() #? Update weights
            scheduler.step()
                
        train_loss = running_loss / len(train_loader)
        # print(f"Epoch: {epoch}, training loss: {train_loss:.4f}")
    
        #* Now we evaluate the model on the validation set, to track training vs validation loss
        model.eval() #? Set model to evaluation mode
        with torch.no_grad(): #? No need to track gradients during evaluation
            val_loss = 0.0    
            for batch, (inputs, labels) in enumerate(val_loader,start=1):#! one pass because val_loader batch size is all, if you want to do it in mini-batches, you MUST change the metric calculations to accept mini-batches
                outputs = model(inputs)
                # labels = labels.cpu() 
                loss = criterion(outputs, labels)
                val_loss += loss.item() #? Calculate loss
            avg_val_loss = val_loss / len(val_loader)
        loss_ratio = val_loss / train_loss    
        pos_weight = loss_ratio  # or any other function of loss_ratio you choose
    
        # Update criterion with new pos_weight
        criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([pos_weight]).to(device))
        # Early stopping
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_model_state = model.state_dict()
            wait = 0
        elif avg_val_loss*0.95 <= best_val_loss:
                wait = 0
        else:
            wait += 1
        if wait >= patience:
            print(f"Early stopping triggered at epoch {epoch}, best val loss: {best_val_loss:.4f}")
            break
        print(f"Epoch: {epoch}".ljust(12), f"training loss:{train_loss:.3f}".ljust(16), f"best_val_loss:{best_val_loss:.3f}".ljust(12), f"Val Loss: {avg_val_loss:.3f}", f"Scheduler lr: {scheduler.get_last_lr()}",end="\r")
    #* Use best model to calculate metrics on the validation set
    #! must be outside epoch loop, it comes after the training and cv loop
    model.load_state_dict(best_model_state) #? Load the best model state
    with torch.no_grad():
        for batch, (inputs, labels) in enumerate(val_loader,start=1):#! one pass because val_loader batch size is all, if you want to do it in mini-batches, you MUST change the metric calculations to accept mini-batches
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                labels = labels.cpu() 
                # predictions = (torch.sigmoid(outputs) < 0.5).float().cpu().numpy()
                predictions = (torch.sigmoid(outputs) >= threshold).float().cpu().numpy()
                val_loss += loss.item() #? Calculate loss
                
    #! The following should have length equal to fold number           
    accuracy=accuracy_score(labels, predictions) 
    precision=precision_score(labels, predictions, pos_label=1, zero_division=0)
    recall=recall_score(labels, predictions, pos_label=1)
    f1=f1_score(labels, predictions, pos_label=1)
    auc=roc_auc_score(labels, predictions)
    
    return model, accuracy, precision, recall, f1, auc


In [ ]:
def maximise_combined_score(trial):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device:", device)
    epochs = 10000
    random_state = 42

    n_neighbours = trial.suggest_int("n_neighbours", 5, 5)
    oversampler = ADASYN(sampling_strategy='minority', n_neighbors=n_neighbours, random_state=random_state)
    
    normalisation_method = trial.suggest_categorical("normalisation_method", ["MinMaxScaler",])
    if normalisation_method:
        if normalisation_method == "MinMaxScaler":
            normalisation_method = MinMaxScaler()
        else:
            normalisation_method = MinMaxScaler()
    contamination = trial.suggest_float("contamination", 0.0, 0.3)
    noise = trial.suggest_float("noise", 0.0, 0.2)
    kFolds = FOLDS_GENERATOR(X_FOR_FOLDS, Y_FOR_FOLDS, 
                         normalisation_method = normalisation_method, 
                         n_splits=5, 
                         oversampler = oversampler, random_state=42, contamination=contamination, noise = noise)
                        
    # Model hyperparameters (first-level optimization)
    hidden_dim = trial.suggest_int("hidden_dim", 32, 128, step = 2)
    hidden_dim2 = trial.suggest_int("hidden_dim2", 32, hidden_dim, step = 2)
    output_dim = trial.suggest_int("output_dim", 64, hidden_dim2, step = 2)
    
    dropout = trial.suggest_float("dropout", 0.0, 0.4)
    threshold = trial.suggest_float("threshold", 0.3, 0.7)
    # dropout = None
    initial_lr = trial.suggest_float("initial_lr", 1e-4, 1e-4, log=True)
    max_lr = trial.suggest_float("max_lr", 1e-4, 1e-4, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)
    
    # Loss function hyperparameters
    # criterion_choice = trial.suggest_categorical("criterion", ["FocalLoss"]) 
    criterion_choice = trial.suggest_categorical("criterion", ["BCEWithLogitsLoss"]) 
    
    # Hyperparameter exploration optimization
    if criterion_choice == "BCEWithLogitsLoss":
        pos_weight = trial.suggest_int("pos_weight", 1, 1)
        alpha = None
        gamma = None
    elif criterion_choice == "FocalLoss":
        pos_weight= None
        alpha = trial.suggest_float("alpha", .25, .25)
        gamma = trial.suggest_float("gamma", .8, .8)
    else:
        pos_weight = None
    
    # Initialize lists for metrics across folds
    accuracy_list = []
    precision_list = []
    recall_list = []
    f1_list = []
    auc_list = []

    # Cross-validation loop
    for fold, (train_x, test_x, train_y, test_y) in enumerate(kFolds, start=1):
        print(f"Fold {fold}:")
        # Create DataLoader for current fold
        train_loader, val_loader = fold_to_dataloader_tensor(train_x, test_x, train_y, test_y, batch_size=64, device=device)
        # Instantiate and initialize the model
        model = MyModel(input_dim=get_feature_count(train_loader), hidden_dim=hidden_dim, hidden_dim2 = hidden_dim2, output_dim=output_dim, dropout=dropout)
        model.to(device)
        model.apply(init_weights)
        
        # Map the choice to the actual loss function
        criterion = criterion_mapping(criterion_choice, pos_weight, alpha, gamma)
        optimiser = optim.Adam(model.parameters(), lr=initial_lr, weight_decay=weight_decay)
        scheduler = torch.optim.lr_scheduler.CyclicLR(optimiser, base_lr=1e-5, max_lr=max_lr)
        
        # Train and evaluate the model on the current fold
        model, accuracy, precision, recall, f1, auc = train_and_evaluate(
            model, criterion, optimiser, scheduler, train_loader, val_loader, epochs=epochs, patience=100, device=device, threshold = threshold
        )
        print(f"Accuracy: {accuracy:.4f}, precision: {precision:.4f}, recall: {recall:.4f}, f1: {f1:.4f}, auc: {auc:.4f}")
        del model
        del train_loader
        del val_loader
    
        # Append the metrics from the current fold
        accuracy_list.append(accuracy)
        precision_list.append(precision)
        recall_list.append(recall)
        f1_list.append(f1)
        auc_list.append(auc)

    # Calculate the average metrics across all folds
    avg_accuracy = np.sum(accuracy_list) / len(accuracy_list)
    avg_precision = np.sum(precision_list) / len(precision_list)
    avg_recall = np.sum(recall_list) / len(recall_list)
    avg_f1 = np.sum(f1_list) / len(f1_list)
    avg_auc = np.sum(auc_list) / len(auc_list)

    # Combine metrics into a single "score"
    # combined_score = (avg_f1 + avg_precision + avg_recall + avg_accuracy + avg_auc) / 5
    combined_score = avg_f1

    return combined_score


In [13]:
import torch
import torch.nn as nn

class FeedForwardBlock(nn.Module):
    def __init__(self, in_features, out_features, dropout=None, activation=nn.ReLU6):
        super().__init__()
        layers = [
            nn.Linear(in_features, out_features),
            nn.BatchNorm1d(out_features),
            activation()
        ]
        if dropout and dropout > 0:
            layers.append(nn.Dropout(dropout))
        self.block = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.block(x)
def print_activations(x, name):
    print(f"{name}: min={x.min().item()}, max={x.max().item()}")
    return x
class MyModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout= None, hidden_dim2= None):
        super().__init__()
        
        # A couple of FeedForward blocks
        self.block1 = FeedForwardBlock(input_dim, hidden_dim, dropout= None)
        self.block2 = FeedForwardBlock(hidden_dim, hidden_dim2, dropout = dropout/2)
        self.block3 = FeedForwardBlock(hidden_dim2, output_dim, dropout)

        # Final output layer (could be softmax, sigmoid, or whatever your target is)
        self.output_layer = nn.Linear(output_dim, 1)  # Just in case you're doing regression or binary classification

    def forward(self, x):
        x = self.block1(x)
        # x = print_activations(x, "after block1")
        x = self.block2(x)
        # x = print_activations(x, "after block2")
        x = self.block3(x)
        # x = print_activations(x, "after block3")
        x = self.output_layer(x)  # Final linear layer
        return x

#* Test the model
test_model = MyModel(input_dim = 20,
                     hidden_dim = 64,
                     hidden_dim2 = 16,
                     output_dim = 4,
                     dropout = .2)
print(test_model)


MyModel(
  (block1): FeedForwardBlock(
    (block): Sequential(
      (0): Linear(in_features=20, out_features=64, bias=True)
      (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6()
    )
  )
  (block2): FeedForwardBlock(
    (block): Sequential(
      (0): Linear(in_features=64, out_features=16, bias=True)
      (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6()
      (3): Dropout(p=0.1, inplace=False)
    )
  )
  (block3): FeedForwardBlock(
    (block): Sequential(
      (0): Linear(in_features=16, out_features=4, bias=True)
      (1): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6()
      (3): Dropout(p=0.2, inplace=False)
    )
  )
  (output_layer): Linear(in_features=4, out_features=1, bias=True)
)


In [8]:
# import torch
# import matplotlib.pyplot as plt

# # Dictionary to store activations for later visualization
# activations = {}

# def hook_fn(module, input, output):
#     # Save the activation output of the layer
#     activations[module] = output.detach()

# # Hook into each of your blocks
# hook1 = model.block1[0].register_forward_hook(hook_fn)  # First Linear Layer in block1
# hook2 = model.block2[0].register_forward_hook(hook_fn)  # First Linear Layer in block2
# hook3 = model.block3[0].register_forward_hook(hook_fn)  # First Linear Layer in block3

# # Example input (assuming your input data is a tensor of the right shape)
# sample_input = torch.randn(1, input_dim)  # Batch size 1, shape (input_dim)

# # Run the input through the model
# _ = model(sample_input)

# # Now you have the activations saved in the activations dictionary
# print(f"Activations for Block 1: {activations[model.block1[0]]}")
# print(f"Activations for Block 2: {activations[model.block2[0]]}")
# print(f"Activations for Block 3: {activations[model.block3[0]]}")

# # Visualize the activations (for the first layer as an example)
# act1 = activations[model.block1[0]].cpu().squeeze()  # Remove batch dimension
# plt.figure(figsize=(8, 8))
# for i in range(min(6, act1.shape[1])):  # Loop over the first few activations (features)
#     plt.subplot(2, 3, i + 1)
#     plt.imshow(act1[0, i].cpu().numpy(), cmap='viridis')  # Visualize first few features
#     plt.axis('off')
# plt.show()

# # Optionally, remove the hooks after visualization to avoid memory leaks
# hook1.remove()
# hook2.remove()
# hook3.remove()


In [21]:
import threading
import optuna
from optuna_dashboard import run_server
# !fuser -k 8080/tcp

# Define your persistent storage
storage = "sqlite:///opt6.db"

# Create or load your study
study_name = "optuna6"
try:
    study = optuna.load_study(study_name=study_name, storage=storage)
except KeyError:
    study = optuna.create_study(study_name=study_name, direction="maximize", storage=storage)

# Start Optuna Dashboard in a separate thread
dashboard_thread = threading.Thread(target=lambda: run_server(storage), daemon=True)
dashboard_thread.start()

# Run optimization
study.optimize(maximise_combined_score, n_trials=100)

# Print results
print("Best trial:")
trial = study.best_trial
print(f"  Combined score: {trial.value}")
print("  Best hyperparameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


Using device: cuda


Bottle v0.13.2 server starting up (using WSGIRefServer())...
Listening on http://localhost:8080/
Hit Ctrl-C to quit.



Fold: 1, Train: (5713, 28), Test: (1149, 28)
Fold: 2, Train: (5855, 28), Test: (1149, 28)
Fold: 3, Train: (5681, 28), Test: (1148, 28)
Fold: 4, Train: (5721, 28), Test: (1148, 28)
Fold: 5, Train: (5698, 28), Test: (1148, 28)
Fold 1:
Early stopping triggered at epoch 143, best val loss: 0.6779s: 0.853 Scheduler lr: [3.870999999999992e-05]]
Accuracy: 0.6475, precision: 0.1919, recall: 0.7759, f1: 0.3077, auc: 0.7045
Fold 2:
Early stopping triggered at epoch 116, best val loss: 0.6824s: 0.949 Scheduler lr: [7.907500000000001e-05]]
Accuracy: 0.6023, precision: 0.1650, recall: 0.7241, f1: 0.2688, auc: 0.6564
Fold 3:
Early stopping triggered at epoch 117, best val loss: 0.5854s: 0.853 Scheduler lr: [9.064e-05]0000004e-05]]
Accuracy: 0.6002, precision: 0.1617, recall: 0.7069, f1: 0.2632, auc: 0.6475
Fold 4:
Early stopping triggered at epoch 121, best val loss: 0.6636s: 0.912 Scheduler lr: [6.940000000000002e-05]]
Accuracy: 0.5993, precision: 0.1654, recall: 0.7328, f1: 0.2698, auc: 0.6585
Fol

[I 2025-04-15 22:32:04,195] Trial 6 finished with value: 0.27741758297159835 and parameters: {'n_neighbours': 5, 'normalisation_method': 'MinMaxScaler', 'contamination': 0.3, 'noise': 0.0, 'hidden_dim': 64, 'hidden_dim2': 32, 'output_dim': 64, 'dropout': 0.2, 'threshold': 0.5, 'initial_lr': 0.0001, 'max_lr': 0.0001, 'weight_decay': 1e-05, 'criterion': 'BCEWithLogitsLoss', 'pos_weight': 1}. Best is trial 6 with value: 0.27741758297159835.


Early stopping triggered at epoch 104, best val loss: 0.5155
Accuracy: 0.6054, precision: 0.1703, recall: 0.7500, f1: 0.2775, auc: 0.6696
Using device: cuda
Fold: 1, Train: (5713, 28), Test: (1149, 28)
Fold: 2, Train: (5855, 28), Test: (1149, 28)
Fold: 3, Train: (5681, 28), Test: (1148, 28)
Fold: 4, Train: (5721, 28), Test: (1148, 28)
Fold: 5, Train: (5698, 28), Test: (1148, 28)
Fold 1:
Early stopping triggered at epoch 126, best val loss: 0.6524s: 0.816 Scheduler lr: [4.9375e-05]000002e-05]]
Accuracy: 0.6388, precision: 0.1866, recall: 0.7672, f1: 0.3002, auc: 0.6958
Fold 2:


[W 2025-04-15 22:33:35,094] Trial 7 failed with parameters: {'n_neighbours': 5, 'normalisation_method': 'MinMaxScaler', 'contamination': 0.3, 'noise': 0.0, 'hidden_dim': 64, 'hidden_dim2': 32, 'output_dim': 64, 'dropout': 0.2, 'threshold': 0.5, 'initial_lr': 0.0001, 'max_lr': 0.0001, 'weight_decay': 1e-05, 'criterion': 'BCEWithLogitsLoss', 'pos_weight': 1} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\TAN LE ZHAN\Documents\GitHub\ADL\.venv\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\TAN LE ZHAN\AppData\Local\Temp\ipykernel_22344\70531313.py", line 74, in maximise_combined_score
    model, accuracy, precision, recall, f1, auc = train_and_evaluate(
                                                  ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\TAN LE ZHAN\AppData\Local\Temp\ipykernel_22344\1161477185.py", line 14, in train_and_eva

KeyboardInterrupt: 